# Time Series Analysis

## Miembros
- Andrés Montoya 21552
- Fernanda Esquivel 21542
- Francisco Castillo 21562
- Fabián Juárez 21440
- Diego Lemus 21469

## Link al repositorio
El repositorio puede ser visualizado [acá](https://github.com/FerEsq/DL-TimeSeries-Analysis)

# Task 1

## Limpieza de datos

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
#Cargar los datos
train_data = pd.read_csv('data/train.csv')

#Convertir la columna 'date' a formato datetime
train_data['date'] = pd.to_datetime(train_data['date'])

#Extraer características de la fecha
train_data['year'] = train_data['date'].dt.year
train_data['month'] = train_data['date'].dt.month
train_data['day_of_week'] = train_data['date'].dt.dayofweek

#Escalar los datos de ventas ('sales') usando StandardScaler
scaler = StandardScaler()
train_data['sales_scaled'] = scaler.fit_transform(train_data[['sales']])

#Verificar los primeros datos transformados
train_data.head()


,date,store,item,sales,year,month,day_of_week,sales_scaled
0,2013-01-01,1,1,13,2013,1,1,-1.362804
1,2013-01-02,1,1,11,2013,1,2,-1.432246
2,2013-01-03,1,1,14,2013,1,3,-1.328083
3,2013-01-04,1,1,13,2013,1,4,-1.362804
4,2013-01-05,1,1,10,2013,1,5,-1.466966


## Preprocesamiento de datos

In [4]:
#Dividir el conjunto de datos en entrenamiento, validación y prueba
train_size = 0.70  # 70% para entrenamiento
validation_size = 0.15  # 15% para validación
test_size = 0.15  # 15% para prueba

total_rows = len(train_data)
train_index = int(total_rows * train_size)
validation_index = int(total_rows * (train_size + validation_size))

train_set = train_data.iloc[:train_index]
validation_set = train_data.iloc[train_index:validation_index]
test_set = train_data.iloc[validation_index:]

In [5]:
#Función para generar secuencias de datos históricos y pronósticos
def create_sequences(data, window_size, forecast_horizon):
    sequences = []
    targets = []
    
    for i in range(len(data) - window_size - forecast_horizon + 1):
        window = data.iloc[i:i + window_size]['sales'].values
        target = data.iloc[i + window_size:i + window_size + forecast_horizon]['sales'].sum()
        sequences.append(window)
        targets.append(target)
    
    return np.array(sequences), np.array(targets)

In [6]:
#Parámetros de secuencias
window_size = 90  #Ventana histórica de 90 días
forecast_horizon = 90  #Pronóstico de 3 meses (90 días)

#Muestra para entrenar y generar secuencias
train_sample = train_set.sample(n=50000, random_state=42)

In [7]:
#Generar secuencias para el conjunto de entrenamiento
train_sequences, train_targets = create_sequences(train_sample, window_size, forecast_horizon)

In [8]:
#Imprimir las dimensiones de las secuencias generadas
print("Dimensiones de las secuencias de entrenamiento:", train_sequences.shape)
print("Dimensiones de los objetivos de entrenamiento:", train_targets.shape)

Dimensiones de las secuencias de entrenamiento: (49821, 90)
Dimensiones de los objetivos de entrenamiento: (49821,)


# Task 2

#### 1. ¿Cuál es el problema del gradiente de fuga en las redes LSTM y cómo afecta la efectividad de LSTM para el pronóstico de series temporales?
* El problema del gradiente de fuga ocurre cuando los gradientes que se propagan a través de las capas de una red neuronal recurrente, como una LSTM, se hacen extremadamente pequeños durante la retropropagación. Esto sucede principalmente en secuencias largas, donde los gradientes se diluyen antes de llegar a las capas iniciales, afectando la capacidad del modelo para aprender dependencias a largo plazo. 
* En el contexto de pronóstico de series temporales, este problema reduce la efectividad de las LSTM al dificultar la captación de patrones o relaciones en series de datos que dependen de eventos distantes en el tiempo. Aunque las LSTM están diseñadas para mitigar este problema mediante su arquitectura de puertas, no lo eliminan completamente, y su desempeño puede verse afectado si las secuencias son demasiado largas o si los datos son ruidosos


#### 2. ¿Cómo se aborda la estacionalidad en los datos de series temporales cuando se utilizan LSTM para realizar pronósticos y qué papel juega la diferenciación en el proceso?
* La estacionalidad en series temporales se refiere a patrones recurrentes que se repiten en intervalos regulares. Para abordar este aspecto al usar LSTM, es común aplicar transformaciones previas a los datos, como la diferenciación estacional, que ayuda a eliminar estos patrones recurrentes antes de entrenar el modelo.
* La diferenciación es una técnica que implica restar valores consecutivos de la serie, ayudando a estabilizar la serie y remover tendencias o estacionalidades. Al hacerlo, el modelo LSTM puede concentrarse en predecir patrones más consistentes, sin ser desviado por ciclos repetitivos. Además de la diferenciación, otras técnicas incluyen el uso de características adicionales (lags) o la incorporación de la información estacional directamente como una entrada al modelo. De esta manera, las LSTM son capaces de enfocarse tanto en las fluctuaciones a corto plazo como en las tendencias subyacentes.


#### 3. ¿Cuál es el concepto de "tamaño de ventana" en el pronóstico de series temporales con LSTM y cómo afecta la elección del tamaño de ventana a la capacidad del modelo para capturar patrones a corto y largo plazo?
* El "tamaño de ventana" se refiere a la cantidad de datos históricos que se utilizan como entrada para el modelo LSTM para predecir el siguiente valor en la serie temporal. Es una ventana deslizante que define el horizonte temporal que el modelo usa para hacer su predicción.
* Si el tamaño de ventana es demasiado pequeño, el modelo puede no tener suficiente información para capturar patrones a largo plazo, lo que afecta su capacidad para hacer predicciones precisas en series con tendencias prolongadas.
* Por otro lado, un tamaño de ventana demasiado grande puede incluir demasiados datos irrelevantes, lo que lleva a un modelo más complejo que podría sobreajustarse a los patrones de ruido en lugar de generalizar. La elección del tamaño de ventana es crucial porque define el equilibrio entre la capacidad del modelo para detectar patrones a corto y largo plazo, y afecta directamente su desempeño en tareas de pronóstico.


### Referencias
1. Buckley, B. (2023). ¿Cuál es el problema del gradiente de fuga? - Academia EITCA. EITCA Academy. https://es.eitca.org/artificial-intelligence/eitc-ai-gcml-google-cloud-machine-learning/first-steps-in-machine-learning/deep-neural-networks-and-estimators/what-is-the-vanishing-gradient-problem/
2. Rojas-Jimenez, K. (s. f.). Capítulo 8 Análisis de Series de Tiempo | Ciencia de Datos para Ciencias Naturales. https://bookdown.org/keilor_rojas/CienciaDatos/an%C3%A1lisis-de-series-de-tiempo.html
3. Andrés, D., & Andrés, D. (2023). Métodos de pronóstico de Series Temporales - ML Pills. ML Pills - Machine Learning Pills. https://mlpills.dev/series-temporales/pronostico-de-series-temporales/